# Verification Experiment for WindLog Sensors

## Introduction
Reliably measuring wind speeds within overwintering groves is an important component of my master's no matter what direction it takes.

Francis has up to 40 anemometers that log data available for my study. The only issue is that they were unreliable in the previous studies they were used. 

I have two on loan which I want to see if they can reliably record measurements. This notebook is the results of that effort. 

## Experimental Design
In a closed room with no open windows or doors, I set up an oscillating fan blowing into the center of the room. The fan is a tower style home fan that is much taller than it is wide. 

I tied a string to the center of the fan and allowed the fan to oscillate all the way to the right. Here I placed one of the windLog instruments (FunStorm) as close as I could to the fan, which was sitting atop my dresser. I tied a knot in the string where the front of the wind instrument was placed. I did this so I could place the other two sensors nearly the same distance away from the fan. 

I allowed the fan to oscillate to the far left, where I placed the second windLog insturment (LlamaLog), being careful to align the instrument so that is the same distance away as FunStorm using the string. I completed the placement by putting the kestrel in the center using the string. 

The length of string that was used is approximately 0.40 m. The height of the sensors was 1.65 m. 

The fan was set to full speed, and the door was closed. The trial period began at 11:35 AM on July 30th, 2023. The trial ended sometime around 5:30 PM the same day. The fan was turned off, but data collection continued while I wrote this and downloaded the data.

All sensors were set to record wind measurements every minute. Both wind log meters were free of any cables during the data collection period. 

Data was downloaded from the windLog instruments using the default software on my work laptop (Windows). At the time of download, Funstorm had 4.6V reading, and LlamaLog had 4.3V.

Curiously, The Funstorm instrument recording a spike in wind measurements when I connected to the computer. This was mirrored in previous experiments. LlamaLog did not have this problem.

## Loading the data

Below I load the measurements from the three datasets and prepare them for comparison. 

###  Kestrel
To start, we'll begin with the Kestrel sensor, which will have it's own peculiarities compared to the WindLog sensors.

In [4]:
import pandas as pd

kestrel = pd.read_csv('/Users/kylenessen/Documents/Code/masters/wind_meter_validation/data/kestrel20230730.csv')
kestrel.head()

ParserError: Error tokenizing data. C error: Expected 2 fields in line 4, saw 16
